In [1]:
%config Completer.use_jedi = False

In [2]:
import numpy as np
import pandas as pd
import preprocessing2
import cosine2
import plotly.express as px

from pdfstructure.hierarchy.parser import HierarchyParser
from pdfstructure.source import FileSource
from pdfstructure.printer import JsonFilePrinter
import pathlib
import json

In [3]:
def text_on_page(dict_var, id_json, list_res, page):
    if type(dict_var) is dict:
        for k, v in dict_var.items():
            if k == id_json and v == page:
                if v > page: return list_res
                list_res.append(dict_var["text"])
            elif isinstance(v, dict):
                text_on_page(v, id_json, list_res, page)   
            elif isinstance(v, list):
                for item in v:
                    text_on_page(item, id_json, list_res, page)
    return list_res

In [4]:
def get_page(data, page):
    lines = []
    for chunk in data["elements"]:
        lines.extend(text_on_page(chunk, "page", [], page))             
    return lines

In [5]:
def get_histogram(docs, top = 20):
    tokens = []
    for s in docs.values():
        tokens += s.split()
    uniques, counts = np.unique(tokens, return_counts = True)
    sorted_inds = np.argsort(counts)
    uniques_sorted = uniques[sorted_inds[-top:]][::-1]
    counts_sorted = counts[sorted_inds[-top:]][::-1]
    return (uniques_sorted, counts_sorted)

In [6]:
file = '/home/vince/Documents/Code/microgoogle/CS4300_microGoogle/pdfs/Nurse.pdf'
start = 1
end = 50

In [7]:
parser = HierarchyParser()
source = FileSource(file, page_numbers=list(range(start-1, end)))

In [8]:
# @st.cache(suppress_st_warning=True)
def fetch_doc(source):
    document = parser.parse_pdf(source)
    printer = JsonFilePrinter()
    file_path = pathlib.Path('pdf.json')
    printer.print(document, file_path=str(file_path.absolute()))
    with open('pdf.json') as json_file:
            data = json.load(json_file)
    pages = {i : get_page(data, i) for i in range(start-1, end)}
    return pages

In [9]:
json = fetch_doc(source)